In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('..'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

../.ipynb_checkpoints/Titanic-checkpoint.ipynb
../Titanic/train.csv
../Titanic/test.csv
../Titanic/Titanic.ipynb


In [3]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [5]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
women = train_data[train_data.Sex == 'female']['Survived']
women_rate = sum(women)/len(women)
men = train_data[train_data.Sex == 'male']['Survived']
men_rate = sum(men)/len(men)
print(women_rate/men_rate)

3.928037164728569


In [59]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
y = train_data['Survived']

features = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked']
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])
X = pd.concat([X, train_data.Age, train_data.Fare], axis = 1)
X_test = pd.concat([X_test, test_data.Age, test_data.Fare], axis = 1)
X.loc[pd.isna(X.Age),'Age'] = X.Age.mean()
X_test.loc[pd.isna(X_test.Age), 'Age'] = X_test.Age.mean()
X.loc[pd.isna(X.Fare),'Fare'] = X.Fare.mean()
X_test.loc[pd.isna(X_test.Fare), 'Fare'] = X_test.Fare.mean()


model = RandomForestClassifier(n_estimators = 5000, max_depth = 7, random_state = 1)
model.fit(X, y)
insample_preds = model.predict(X)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived':predictions})
output.to_csv('RF_sub.csv', index = False)

print(classification_report(y_true = y, y_pred=insample_preds))

              precision    recall  f1-score   support

           0       0.88      0.96      0.92       549
           1       0.93      0.78      0.85       342

    accuracy                           0.89       891
   macro avg       0.90      0.87      0.88       891
weighted avg       0.90      0.89      0.89       891



In [62]:
from sklearn.ensemble import GradientBoostingClassifier

gb_model = GradientBoostingClassifier(n_estimators=500, random_state= 1, loss = 'exponential', subsample=0.5)
gb_model.fit(X,y)

gb_insample_pred = gb_model.predict(X)
gb_pred = gb_model.predict(X_test)

gb_output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived':gb_pred})
gb_output.to_csv('GB_sub.csv', index = False)

print(classification_report(y_true = y, y_pred=gb_insample_pred))

              precision    recall  f1-score   support

           0       0.94      0.98      0.96       549
           1       0.96      0.90      0.93       342

    accuracy                           0.95       891
   macro avg       0.95      0.94      0.94       891
weighted avg       0.95      0.95      0.95       891



,Pclass,SibSp,Parch,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Age,Age,Fare
0,3,1,0,0,1,0,0,1,22.000000,22.0,7.2500
1,1,1,0,1,0,1,0,0,38.000000,38.0,71.2833
2,3,0,0,1,0,0,0,1,26.000000,26.0,7.9250
3,1,1,0,1,0,0,0,1,35.000000,35.0,53.1000
4,3,0,0,0,1,0,0,1,35.000000,35.0,8.0500
5,3,0,0,0,1,0,1,0,29.699118,NaN,8.4583
6,1,0,0,0,1,0,0,1,54.000000,54.0,51.8625
7,3,3,1,0,1,0,0,1,2.000000,2.0,21.0750
8,3,0,2,1,0,0,0,1,27.000000,27.0,11.1333
9,2,1,0,1,0,1,0,0,14.000000,14.0,30.0708


In [31]:
train_data.Age

0      22.0
1      38.0
2      26.0
3      35.0
4      35.0
5       NaN
6      54.0
7       2.0
8      27.0
9      14.0
10      4.0
11     58.0
12     20.0
13     39.0
14     14.0
15     55.0
16      2.0
17      NaN
18     31.0
19      NaN
20     35.0
21     34.0
22     15.0
23     28.0
24      8.0
25     38.0
26      NaN
27     19.0
28      NaN
29      NaN
       ... 
861    21.0
862    48.0
863     NaN
864    24.0
865    42.0
866    27.0
867    31.0
868     NaN
869     4.0
870    26.0
871    47.0
872    33.0
873    47.0
874    28.0
875    15.0
876    20.0
877    19.0
878     NaN
879    56.0
880    25.0
881    33.0
882    22.0
883    28.0
884    25.0
885    39.0
886    27.0
887    19.0
888     NaN
889    26.0
890    32.0
Name: Age, Length: 891, dtype: float64